In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

from tensorflow.keras.datasets import mnist
import keras

BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/"

In [2]:
# Generate the train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
 
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
 
# Generate some noisy data
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
 
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

11493376/11490434 [==============================] - 6s 1us/step


In [7]:
# sudden noisy dataset
x_train_sudden = np.vstack((x_train[:30000], x_train[:30000] + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train[:30000].shape)))
x_test_sudden = np.vstack((x_test[:30000], x_test[:30000] + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test[:30000].shape)))

x_train_sudden = np.clip(x_train_sudden, 0., 1.)
x_test_sudden = np.clip(x_test_sudden, 0., 1.)

In [10]:
# # incremental noisy dataset

act_res = []
po = np.linspace(0, 1, x_train.shape[0])
for i in range(x_train.shape[0]):
    act_res.append(np.random.choice(2, 1, p=[1-po[i], po[i]])[0])

noise = noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
for i in range(x_train.shape[0]):
    noise[i] *= act_res[i]

x_train_inc = x_train + noise
x_train_inc = np.clip(x_train_inc, 0., 1.)

act_res = []
po = np.linspace(0, 1, x_test.shape[0])
for i in range(x_test.shape[0]):
    act_res.append(np.random.choice(2, 1, p=[1-po[i], po[i]])[0])

noise = noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
for i in range(x_test.shape[0]):
    noise[i] *= act_res[i]

x_test_inc = x_test + noise
x_test_inc = np.clip(x_test_inc, 0., 1.)

In [9]:
# # gradual noisy dataset

x = np.linspace(-2*np.pi, 2*np.pi, int(x_train.shape[0] / 2))
arr = np.exp2(x)
start = 0
end = 1
width = end - start
res = (arr - arr.min())/(arr.max() - arr.min()) * width + start
res[res>0.5] = 0.5
act_res = np.ones(x_train.shape[0])
act_res[:int(x_train.shape[0] / 2)] = res
act_res[int(x_train.shape[0] / 2):] = 0.5

noise = noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
for i in range(x_train.shape[0]):
    noise[i] *= act_res[i]

x_train_grad = x_train + noise
x_train_grad = np.clip(x_train_grad, 0., 1.)

x = np.linspace(-2*np.pi, 2*np.pi, int(x_test.shape[0] / 2))
arr = np.exp2(x)
start = 0
end = 1
width = end - start
res = (arr - arr.min())/(arr.max() - arr.min()) * width + start
res[res>0.5] = 0.5
act_res = np.ones(x_test.shape[0])
act_res[:int(x_test.shape[0] / 2)] = res
act_res[int(x_test.shape[0] / 2):] = 0.5

noise = noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
for i in range(x_test.shape[0]):
    noise[i] *= act_res[i]


x_test_grad = x_test + noise
x_test_grad = np.clip(x_test_grad, 0., 1.)

In [8]:
def make_model():
    model = Sequential()

    model.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(32,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(64,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

    return model

def train_model(model, x_train, y_train, name):
    filepath = BASE_PATH + name + "-{epoch:02d}"
    checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=False, mode='auto', save_freq='epoch')
    epochs = 45
    annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x+epochs))
    model.fit(x_train, y_train, batch_size=64, epochs = epochs, steps_per_epoch = x_train.shape[0]//64, callbacks=[annealer, checkpoint])

In [9]:
model = make_model()
model.save(BASE_PATH + 'base_model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/base_model/assets


Experiment 1 - full mnist model

In [10]:
m1 = tf.keras.models.load_model(BASE_PATH + 'base_model')
train_model(m1, x_train, y_train, 'based_model')
results = m1.evaluate(x_test, y_test)
results = m1.evaluate(x_test_noisy, y_test)

Epoch 1/45
937/937 [==============================] - 8s 7ms/step - loss: 0.8133 - accuracy: 0.7470

Epoch 00001: saving model to /content/drive/MyDrive/Colab Notebooks/based_model-01
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/based_model-01/assets
Epoch 2/45
937/937 [==============================] - 7s 7ms/step - loss: 0.1874 - accuracy: 0.9426

Epoch 00002: saving model to /content/drive/MyDrive/Colab Notebooks/based_model-02
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/based_model-02/assets
Epoch 3/45
937/937 [==============================] - 7s 7ms/step - loss: 0.1197 - accuracy: 0.9634

Epoch 00003: saving model to /content/drive/MyDrive/Colab Notebooks/based_model-03
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/based_model-03/assets
Epoch 4/45
937/937 [==============================] - 7s 7ms/step - loss: 0.0889 - accuracy: 0.9732

Epoch 00004: saving model to /content/drive/MyDrive/Colab N

Experiment 2 - Sudden drift of 30k

In [12]:
m2 = tf.keras.models.load_model(BASE_PATH + 'based_model-45')
train_model(m2, x_train_noisy[:30000], y_train[:30000], 'sudden_drift_30000')
results = m2.evaluate(x_test, y_test)
results = m2.evaluate(x_test_noisy, y_test)

Epoch 1/45
468/468 [==============================] - 4s 7ms/step - loss: 0.2510 - accuracy: 0.9310

Epoch 00001: saving model to /content/drive/MyDrive/Colab Notebooks/sudden_drift_30000-01
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/sudden_drift_30000-01/assets
Epoch 2/45
468/468 [==============================] - 4s 7ms/step - loss: 0.1506 - accuracy: 0.9549

Epoch 00002: saving model to /content/drive/MyDrive/Colab Notebooks/sudden_drift_30000-02
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/sudden_drift_30000-02/assets
Epoch 3/45
468/468 [==============================] - 3s 7ms/step - loss: 0.1207 - accuracy: 0.9606

Epoch 00003: saving model to /content/drive/MyDrive/Colab Notebooks/sudden_drift_30000-03
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/sudden_drift_30000-03/assets
Epoch 4/45
468/468 [==============================] - 3s 7ms/step - loss: 0.1071 - accuracy: 0.9656

Epoch 00004: savi

Experiment 3 - train full mnsit and 30k drift together

In [14]:
m3 = tf.keras.models.load_model(BASE_PATH + 'base_model')
train_model(m3, np.vstack((x_train, x_train_noisy[:30000])), np.vstack((y_train, y_train[:30000])), 'mnist_and_30k')
results = m3.evaluate(x_test, y_test)
results = m3.evaluate(x_test_noisy, y_test)

Epoch 1/45
1406/1406 [==============================] - 11s 7ms/step - loss: 0.8734 - accuracy: 0.7287

Epoch 00001: saving model to /content/drive/MyDrive/Colab Notebooks/mnist_and_30k-01
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/mnist_and_30k-01/assets
Epoch 2/45
1406/1406 [==============================] - 10s 7ms/step - loss: 0.2316 - accuracy: 0.9282

Epoch 00002: saving model to /content/drive/MyDrive/Colab Notebooks/mnist_and_30k-02
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/mnist_and_30k-02/assets
Epoch 3/45
1406/1406 [==============================] - 10s 7ms/step - loss: 0.1493 - accuracy: 0.9529

Epoch 00003: saving model to /content/drive/MyDrive/Colab Notebooks/mnist_and_30k-03
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/mnist_and_30k-03/assets
Epoch 4/45
1406/1406 [==============================] - 10s 7ms/step - loss: 0.1144 - accuracy: 0.9641

Epoch 00004: saving model to /conte

Experiment 4 - half mnist + 10k mixed data

In [15]:
k1 = tf.keras.models.load_model(BASE_PATH + 'base_model')
train_model(k1, x_train[:30000], y_train[:30000], '30k_normal')
results = k1.evaluate(x_test, y_test)
results = k1.evaluate(x_test_noisy, y_test)

Epoch 1/45
468/468 [==============================] - 4s 7ms/step - loss: 1.2525 - accuracy: 0.6096

Epoch 00001: saving model to /content/drive/MyDrive/Colab Notebooks/30k_normal-01
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/30k_normal-01/assets
Epoch 2/45
468/468 [==============================] - 3s 7ms/step - loss: 0.3624 - accuracy: 0.8873

Epoch 00002: saving model to /content/drive/MyDrive/Colab Notebooks/30k_normal-02
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/30k_normal-02/assets
Epoch 3/45
468/468 [==============================] - 3s 7ms/step - loss: 0.2217 - accuracy: 0.9328

Epoch 00003: saving model to /content/drive/MyDrive/Colab Notebooks/30k_normal-03
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/30k_normal-03/assets
Epoch 4/45
468/468 [==============================] - 3s 7ms/step - loss: 0.1569 - accuracy: 0.9531

Epoch 00004: saving model to /content/drive/MyDrive/Colab Noteboo

In [16]:
x_gc_train = np.vstack((x_train[30000:35000], x_train_noisy[:5000]))
y_gc_train = np.vstack((y_train[30000:35000], y_train[:5000]))

idx = [i for i in range(len(x_gc_train))]
np.random.shuffle(idx)

x_gc_train = x_gc_train[idx]
y_gc_train = y_gc_train[idx]

x_gc_train.shape
y_gc_train.shape

(10000, 10)

In [17]:
train_model(k1, x_gc_train, y_gc_train, '30k_normal_added_10k_mix')
results = k1.evaluate(x_test, y_test)
results = k1.evaluate(x_test_noisy, y_test)

Epoch 1/45
156/156 [==============================] - 1s 8ms/step - loss: 0.2115 - accuracy: 0.9365

Epoch 00001: saving model to /content/drive/MyDrive/Colab Notebooks/30k_normal_added_10k_mix-01
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/30k_normal_added_10k_mix-01/assets
Epoch 2/45
156/156 [==============================] - 1s 8ms/step - loss: 0.1164 - accuracy: 0.9631

Epoch 00002: saving model to /content/drive/MyDrive/Colab Notebooks/30k_normal_added_10k_mix-02
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/30k_normal_added_10k_mix-02/assets
Epoch 3/45
156/156 [==============================] - 1s 8ms/step - loss: 0.0993 - accuracy: 0.9669

Epoch 00003: saving model to /content/drive/MyDrive/Colab Notebooks/30k_normal_added_10k_mix-03
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/30k_normal_added_10k_mix-03/assets
Epoch 4/45
156/156 [==============================] - 1s 8ms/step - loss: 0.0897 -